## Problem Set 2: Merging and regular expressions

**Total points (without extra credit)**: 30 

**Background on the policy context**: here, we're going to use two datasets to practice reshaping, merging, and regular expression patterns. Both datasets relate to the broader issue of which employers might be violating the rights of temporary guestworkers granted visas under the H-2A program. Here are some articles about potential exploitation of guestworkers by firms and inequality caused by minimal oversight:

- News media coverage of labor abuses of temporary guestworkers: https://www.buzzfeednews.com/article/kenbensinger/the-pushovers 
- GAO report on labor abuses of temporary guestworkers: https://www.gao.gov/products/gao-15-154

The following datasets are located in `pset2_inputdata` (need to unzip): 

- `jobs_clean`: a dataset of guestworker jobs posted by many employers, some of whom have been debarred (banned) from the program for labor abuses; others not debarred
- `debar`: a dataset of employers who committed violations of labor regulations meant to protect temporary guestworkers 


You can view a codebook here: https://docs.google.com/spreadsheets/d/1rF9GJEC8pPKxipD0TsoG9DVdqz3EJ-b-BHEtyioAX7I/edit?usp=sharing


In [1]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# 1. Reshaping data (13 points total)

Load the following dataset stored in `pset2_inputdata`: `debar.csv`

This represents employers temporarily banned from hiring workers (debar.csv); call this `debar`


View the head()


In [7]:
debar = pd.read_csv("C:/Users/ichir/Documents/GitHub/QSS20_S24_Sankey/problemsets/pset2/pset2_inputdata/debar.csv")

## 1.1 (1 point)

Print the number of rows in `debar` versus the number of unique employer names (`Name`). Is there one row per employer or multiple rows for some employers?

In [8]:
## your code here
len(debar.index)
debar["Name"].nunique()

# There are 114 rows and 98 unique employers
# There are multiple rows for some emplyees which explains the differerene

114

98

## 1.2 Investigating duplicated rows (2 points)

A. Create a new column in `debar`--`is_repeated`-- that tells us whether an employer (`Name`) is repeated > 1 times

*Hint*: there are multiple ways to solve this but some possibilities to get the list of names that are repeated are:
- Using value_counts() on the `Name` variable and extracting the index from that value counts 
- Using groupby to count the rows attached to one name

B. Print the rows where `is_repeated == True` and interpret

C. Subset to the rows where `is_repeated == True` and save that data as `mult_debar`. Print the head() and shape

In [9]:
# Count the occurrences of each name and identify names that are repeated more than once
repeated_names = debar['Name'].value_counts()
repeated_names = repeated_names[repeated_names > 1].index

# Create a new column 'is_repeated' based on whether the 'Name' appears in the list of repeated names
debar['is_repeated'] = debar['Name'].isin(repeated_names)

# Subset these rows to a new DataFrame and inspect its contents
mult_debar = debar[debar['is_repeated'] == True]
mult_debar

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,True
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,True
11,Loewen Harvesting LLC,"Brownsville, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,True
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,True
14,Sharon Mathis,"Tifton, GA",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
15,SRT Farms,"Morton, TX",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
16,Mark Duncan,"Roosevelt, UT",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
17,"Maple Ridge Custom Services, LLC","Altheimer, AK",Failure to respond to audit (partial response),2 years,11/16/2014,11/15/2016,True
18,F&W Farms,"Ingalls, KS",Failure to respond to audit (partial response),2 years,12/10/2014,12/9/2016,True


In [46]:
mult_debar.head()
mult_debar.shape

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,True
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,True
11,Loewen Harvesting LLC,"Brownsville, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,True
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,True


(32, 7)

## 1.3 Reshape mult_debar to wide to begin filtering out duplicates (4 points)

You want to separate out two cases:

- Cases where the repeat rows for one employer are due to duplicated data 
- Cases where the repeat rows for one employer represent repeated violations for different issues

There are various ways to check duplicates in this data (eg converting `Violation` to lowercase; replacing spelled-out states with two-dig state codes)

We're going to use the simple rule of:

- A row is a duplicate if, within an employer (defined by Name + City, State), the Start date for each row's violation is the same 

To begin to check this, reshape `mult_debar` to a wide dataframe (`mult_debar_wide`) with the following columns, treating the `Name` and `City, State` as the index for the pivot:

- Name
- City, State
- start_date_viol1
- start_date_viol2

Print the head and shape

In [ ]:
## your code here
mult_debar.loc[:, 'Violation Count'] = mult_debar.groupby(['Name']).cumcount() + 1

# Pivot the data
mult_debar_wide = mult_debar.pivot_table(index=['Name'],
                                         columns='Violation Count',
                                         values='Start date',
                                         aggfunc='first')

# Rename columns to start_date_viol1, start_date_viol2, etc.
mult_debar_wide.columns = [f'start_date_viol{col}' for col in mult_debar_wide.columns]

# Reset index if needed to turn index back to columns
mult_debar_wide.reset_index(inplace=True)


mult_debar_wide
mult_debar.head()
mult_debar_wide.shape

C:\Users\ichir\AppData\Local\Temp\ipykernel_4852\3213388747.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mult_debar.loc[:, 'Violation Count'] = mult_debar.groupby(['Name']).cumcount() + 1


## 1.4 Filter out duplicates from original debar data (6 points)

A. Using `mult_debar_wide`, add a column `is_dup` that takes value of True for cases where start_date_viol1 == start_date_viol2 marking the row as a duplicate

B. Going back to the original long-format data you loaded at the beginning- `debar`
    - For employers where `is_dup == True` as indicated by your wide-format dataframe, only keep `violnum == viol1`
    - For all other employers (so is_dup == False and ones we didnt need to check duplicates for), keep all violnum
    - Remove the `is_repeated` column from the `debar` data

**Hint**: you can complete part B without a for loop; `pd.concat` with axis = 0 (row binding) is one way

Call the resulting dataframe `debar_clean` and print the shape and # of unique employer names

In [58]:
# A. add 'is_dup' column
mult_debar_wide['is_dup'] = mult_debar_wide['start_date_viol1'] == mult_debar_wide['start_date_viol2']


# B. creating a dictionary where keys are the index (Name) from mult_debar_wide
# and values are the is_dup status, calling map(dup_mapping)
# to check each row in debar checks its Name against this dictionary and gets is_dup status.
dup_mapping = mult_debar_wide.set_index(['Name'])['is_dup']

# Apply this mapping to the 'debar' dataset
debar['is_dup'] = debar.set_index(['Name']).index.map(dup_mapping)

# Combine the rows that need to be filtered and those that do not need filtering
# Filter duplicates where is_dup is True and keep only the first occurrence
duplicates_filtered = debar[debar['is_dup'] == True].drop_duplicates(subset=['Name'], keep='first')

# Get all entries where either is_dup is False or NaN (not checked for duplicates)
not_duplicates = debar[debar['is_dup'] != True]

# Combine the two dataframes
debar_clean = pd.concat([duplicates_filtered, not_duplicates], axis=0)

# Remove 'is_repeated' and 'is_dup' columns
debar_clean.drop(columns=['is_repeated', 'is_dup'], inplace=True)

# Print the shape and the number of unique employer names
debar_clean.shape
debar_clean['Name'].nunique()

(101, 6)

98

# 2. Merging and regex (17 points total)



##2.1 Load data on job postings

The previous dataset contains a small subset of employers who faced temporary bans due to violations of H-2A program regulations

Since most of the bans have expired, we're going to see which of those employers posted new H-2A jobs in the first quarter of 2021 

Loading the `jobs_clean.csv` data stored in `pset4_inputdata`

In [59]:
jobs = pd.read_csv("C:/Users/User/Documents/GitHub/QSS20_S24/problemsets/pset2/pset2_inputdata/jobs.csv")
jobs.head()

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,ADDENDUM_B_HOUSING_ATTACHED,TOTAL_HOUSING_RECORDS,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS
0,H-300-20199-721302,Determination Issued - Withdrawn,2020-07-17 14:50:40.840,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,Y,"Fazio Farms Operating Company, LLC",NaN,...,N,1,Y,12.68,12.68,55.0,13607017661,faziofarms@gmail.com,NaN,0
1,H-300-20231-773906,Determination Issued - Certification,2020-08-20 10:38:15.620,2020-10-01 00:00:00.000,Association - Agent,N,Seasonal,N,Charlie Sunderland,Panter & Sunderland Nursery,...,N,1,N,NaN,12.68,55.0,19318083783,NaN,https://www.jobs4tn.gov/vosnet/Default.aspx,0
2,H-300-20231-774123,Determination Issued - Certification,2020-08-24 15:33:14.340,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,Michael Rudebusch,NaN,...,N,1,N,NaN,12.68,55.0,19369333827,fayethlynpitre@rocketmail.com,NaN,0
3,H-300-20231-774151,Determination Issued - Certification,2020-08-21 12:08:09.760,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,Lodahl Farms,NaN,...,Y,2,N,NaN,12.68,55.0,14069637560,lodahl_kelsey@yahoo.com,NaN,0
4,H-300-20231-774508,Determination Issued - Certification,2020-08-20 10:17:34.530,2020-10-01 00:00:00.000,Individual Employer,Y,Seasonal,N,"Dunson Harvesting, Inc.","Dunson Harvesting, Inc.",...,Y,8,N,NaN,12.68,55.0,18632939888,NaN,www.employflorida.com,4


##  2.2 Try inner join on employer name  (2 points)

- Use the `EMPLOYER_NAME` field of the `jobs` dataset
- Use the `Name` field of the `debar_clean` dataset 

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are exact matches, print the row(s) with exact matches



In [60]:
matches = pd.merge(jobs, debar_clean, left_on='EMPLOYER_NAME', right_on='Name', how='inner')

matches
#one match

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS,Name,"City, State",Violation,Duration,Start date,End date
0,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,Rafael Barajas,NaN,...,18632732686,NaN,https://seasonaljobs.dol.gov/,7,Rafael Barajas,"Sebring, Florida",Non-payment of certification fee,1 year,9/23/2016,9/22/2017


## 2.3 Targeted regex (10 points total)

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the employer name fields in each dataset 

### 2.3.1 Converting to upper (2 points)

A. Convert the `EMPLOYER_NAME` and `Name` fields to uppercase using list comprehension rather than df.varname.str.upper() (it's fine to do a separate list comprehension line for each of the two columns)

B. Print a random sample of 15 values of each result

C. Assign the full vector of uppercase names back to the original data, writing over the original `EMPLOYER_NAME` and `Name` columns 


In [62]:
## insert your code to turn into uppercase here
# Convert 'EMPLOYER_NAME' in jobs DataFrame to uppercase
jobs['EMPLOYER_NAME'] = jobs['EMPLOYER_NAME'].apply(lambda name: name.upper())

# Convert 'Name' in debar_clean DataFrame to uppercase
debar_clean['Name'] = debar_clean['Name'].apply(lambda name: name.upper())

In [63]:
## insert your code for the random sample
jobs["EMPLOYER_NAME"].sample(n=15)
debar_clean["Name"].sample(n=15)

33                    PHIL WALTERS
1218                 BARTON SCHOTT
1136             SESTER FARMS, INC
2182     DICK CLARK LIVESTOCK, LLC
603      ROLLING CREEK PARTNERSHIP
905              CLINT FREYOU FARM
681             JARED BENTON FARMS
2169           BONNIE PLANTS, INC.
1228             TIQUA MANUEL FARM
1853      FRANCIS ADRIAN AUGUSTINE
492               JOSE H. MARTINEZ
2550                  VANCE FISHER
2556    ROCKY CREEK NURSERIES INC.
2634               DELTA FARMS INC
146                 EAT RICE, INC.
Name: EMPLOYER_NAME, dtype: object

63                           RAFAEL BARAJAS
23                            REIMER’S INC.
78                          OMEGA LAMB, LLC
4          GREAT PLAINS FLUID SERVICE, INC.
42                      LEONARD SMITH FARMS
90           RUBEN RUIZ (DESOTO HARVESTING)
107                            WALKER PLACE
74     JIM AND ANN SHIPLEY WILLIAM SHIPLEY*
81                              OLSON FARMS
13                            REIMER'S INC.
40                          JEREMY CHANDLER
69                           QUENTIN SCHELL
32                   FIRST AMERICAN HOLDING
41                          K W ENTERPRISES
80                          M KENESTON CORP
Name: Name, dtype: object

In [64]:
## insert your code for assigning the uppercase names back to the data
# Overwrite 'EMPLOYER_NAME' in jobs DataFrame with its uppercase version
jobs['EMPLOYER_NAME'] = jobs['EMPLOYER_NAME'].str.upper()

# Overwrite 'Name' in debar_clean DataFrame with its uppercase version
debar_clean['Name'] = debar_clean['Name'].str.upper()


### 2.3.2 Cleaning up punctuation (4 points)

You notice that INC, CO, and LLC are sometimes followed by a period (.) but sometimes not

A. For each dataset, write a regex pattern using `re.sub` to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example we provide below and print the result. See the Github issue for examples of what to return


**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content



In [65]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [66]:
## insert your code here with the regex pattern for part A
def remove_period(input_string):
    pattern = r"(INC|CO|LLC)\."
    return re.sub(pattern, r"\1", input_string)

## insert your code to use re.sub to apply the pattern to the test cases for part B
print("Positive example 1 Old:", pos_example_1, "New:", remove_period(pos_example_1))
print("Positive example 2 Old:", pos_example_2, "New:", remove_period(pos_example_2))
print("Negative example Old:", neg_example, "New:", remove_period(neg_example))

Positive example 1 Old: CISCO PRODUCE INC. New: CISCO PRODUCE INC
Positive example 2 Old: AVOYELLES HONEY CO., LLC New: AVOYELLES HONEY CO, LLC
Negative example Old: E.V. RANCH LLP New: E.V. RANCH LLP


### 2.3.3 (4 points)

Use that pattern in conjunction with `re.sub` and list comprehension to clean the employer name columns in each dataset. Save the new columns as `name_clean` in each. Then, use row subsetting to (1) subset to rows that changed names and (2) for:

- `debar_clean` print the `Name` and `name_clean` columns
- `jobs` print the `EMPLOYER_NAME` and `name_clean` columns

Make sure to use the uppercase versions of the variables


In [67]:
## your code here to clean the columns

def clean_name(name):
    pattern = r'\b(INC|CO|LLC)\.'
    return re.sub(pattern, r'\1', name)

# Apply the cleaning function to each dataset
jobs['name_clean'] = [clean_name(name) for name in jobs['EMPLOYER_NAME'].str.upper()]
debar_clean['name_clean'] = [clean_name(name) for name in debar_clean['Name'].str.upper()]

# Subset and print changes
changed_jobs = jobs[jobs['EMPLOYER_NAME'].str.upper() != jobs['name_clean']]
changed_debar = debar_clean[debar_clean['Name'].str.upper() != debar_clean['name_clean']]

print("Changes in debar_clean:")
print(changed_debar[['Name', 'name_clean']])

Changes in debar_clean:
                                Name                       name_clean
3              ANTON FERTILIZER INC.             ANTON FERTILIZER INC
4   GREAT PLAINS FLUID SERVICE, INC.  GREAT PLAINS FLUID SERVICE, INC
5                        PROMAX INC.                       PROMAX INC
13                     REIMER'S INC.                     REIMER'S INC
19                CISCO PRODUCE INC.                CISCO PRODUCE INC
23                     REIMER’S INC.                     REIMER’S INC
33                GERONIMO SHEEP CO.                GERONIMO SHEEP CO
46          ALTENDORF TRANSPORT INC.          ALTENDORF TRANSPORT INC
56                CISCO PRODUCE INC.                CISCO PRODUCE INC
72      SAXTONS RIVER ORCHARDS, INC.      SAXTONS RIVER ORCHARDS, INC
91          AVOYELLES HONEY CO., LLC          AVOYELLES HONEY CO, LLC


In [68]:
## your code here to print the head
changed_debar["name_clean"].head()
changed_jobs["name_clean"].head()

3                ANTON FERTILIZER INC
4     GREAT PLAINS FLUID SERVICE, INC
5                          PROMAX INC
13                       REIMER'S INC
19                  CISCO PRODUCE INC
Name: name_clean, dtype: object

4                      DUNSON HARVESTING, INC
7     FARM LABOR ASSOCIATION FOR GROWERS, INC
14                          MCLAIN FARMS, INC
17                         BONNIE PLANTS, INC
18                 B & W QUALITY GROWERS, INC
Name: name_clean, dtype: object

## 2.4 More joins and more cleaning (5 points)

A. Conduct another inner join between `jobs` and `debar_clean` now using the `name_clean` column; print the result. Did the cleaning result in any more employers matched between the two datasets?

B. Create a new column in `debar_clean` called `name_clean_2` that uses regex to take the following name in that dataset:

- `SLASH E.V. RANCH LLP` in the `debar_clean` dataset

And cleans it up so that it matches with this employer in `jobs`

- `SLASH EV RANCH` in the `jobs` dataset

Eg a pattern to remove the dots in the EV and the space+LLP-- you can apply the pattern to all employer names in debar_clean (so don't need to worry about only applying it to that one employer)


C. Conduct a left join using `name_clean_2` as the join column where the left hand dataframe is `jobs`; right hand dataframe is `debar_clean`, store the result as a dataframe, and print the rows where the merge indicator indicates the row was found in both dataframe

**Note**: this manual cleaning process is inefficient and helps motivate why talked about fuzzy matching. Fuzzy matching could recognize that Slash EV ranch is a highly similar string to slash ev ranch llp and match them without us needing to use regex to make the strings identical.

In [69]:
## your code here
# A
inner_join_a = pd.merge(jobs, debar_clean, left_on='name_clean', right_on='name_clean', how='inner')

inner_join_a
# There are not any more employers matches between the two datasets.

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS,name_clean,Name,"City, State",Violation,Duration,Start date,End date
0,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,RAFAEL BARAJAS,NaN,...,NaN,https://seasonaljobs.dol.gov/,7,RAFAEL BARAJAS,RAFAEL BARAJAS,"Sebring, Florida",Non-payment of certification fee,1 year,9/23/2016,9/22/2017


In [70]:
# B
def clean_names(name):
    # Remove dots after initials and spaces before LLP
    name = re.sub(r'\b(\w)\.(\w)\.\b', r'\1\2', name)  # Converts 'E.V.' to 'EV'
    name = re.sub(r'\s+LLP\b', '', name)  # Removes ' LLP'
    return name

# Apply the cleaning function to create the new column
debar_clean['name_clean_2'] = debar_clean['Name'].apply(clean_names)

# Print the updated DataFrame to check the new column
print(debar_clean[['Name', 'name_clean_2']])

                                                  Name  \
6                              ANNABELLA LAND & CATTLE   
7                                  AUTUMN HILL ORCHARD   
8                CADDO CREEK RANCH, DBA PARADISE RANCH   
11                               LOEWEN HARVESTING LLC   
12                         ROLLO FARM LABOR CONTRACTOR   
..                                                 ...   
107                                       WALKER PLACE   
108  COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...   
110                                      JESUS LEDESMA   
112        B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL)   
113                                        DELIA ROJAS   

                                          name_clean_2  
6                              ANNABELLA LAND & CATTLE  
7                                  AUTUMN HILL ORCHARD  
8                CADDO CREEK RANCH, DBA PARADISE RANCH  
11                               LOEWEN HARVESTING LLC  
12                

In [71]:
# C
merged_data = pd.merge(jobs, debar_clean, left_on='name_clean', right_on='name_clean_2', how='inner', indicator=True)

# Filter and print rows where the merge indicator is 'both', indicating presence in both DataFrames
matched_rows = merged_data[merged_data['_merge'] == 'both']
matched_rows

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,name_clean_x,Name,"City, State",Violation,Duration,Start date,End date,name_clean_y,name_clean_2,_merge
0,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,RAFAEL BARAJAS,NaN,...,RAFAEL BARAJAS,RAFAEL BARAJAS,"Sebring, Florida",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,RAFAEL BARAJAS,RAFAEL BARAJAS,both


# 3. Optional extra credit 1: regex to separate companies from individuals (1 point)

You notice some employers in `debar_clean` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned `name_clean` in `debar_clean`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string; so in example above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the "and")
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
**Hints and resources**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

- Same regex resources as above
    

In [ ]:
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"
neg_example = "CISCO PRODUCE INC"

## your code here to define the pattern

## your code here to apply it to the pos_example

## your code here to apply it to the negative example

C. Iterate over the `name_clean` column in debar and use regex to create two new columns in `debar_clean`:
   - `co_name`: A column for company (full `name_clean` string if no match; pattern before COMPANY if one extracted)
   - `ind_name`: A column for individual (full `name_clean` string if no match; pattern before INDIVIDUAL if one extracted)
 


In [ ]:
# your code here

   
D. Print three columns for the rows in `debar_clean` containing the negative example and positive example described above (county fair farm and cisco produce):

- `name_clean`
- `co_name`
- `ind_name`
- `Violation`

**Note**: as shown in the outcome there may be duplicates of the same company reflecting different violations

In [ ]:
# your code here

# 4. Optional extra credit 2 (up to 3 points)

- For 1 point extra credit, create a visualization with 1+ of the existing fields in either the raw `jobs` or `debar` data. We'll be showing cool visualizations in class so use your imagination! Options could include visualizing between-state or over-time variation

- For 3 points extra credit instead, geocode the employer addresses in `jobs` and plot the addresses of jobs as points overlaid on top of a map of Georgia 
    - **Note**: this extra credit involves Googling since we have not yet covered spatial data. 
        - For discussion of how to geocode addresses -> lat/long, see: https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/ 
        - For discussion of plotting lat/long dots against a map, see this discussion of geopandas: https://towardsdatascience.com/plotting-maps-with-geopandas-428c97295a73
    - Relevant columns include `EMPLOYER_ADDRESS_1` 
    - The geocoding might have a long runtime so feel free to implement it in a separate .py script that you submit alongside your notebook and to just read in the geocoded data

In [ ]:
## your code here